<a href="https://colab.research.google.com/github/holly-hewitt/Abnormal-Infant-Movement-Detection/blob/main/Code/AbnormalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import tensorflow.compat.v1 as tf
#tf.enable_eager_execution(tf.ConfigProto(log_device_placement=False))
#tf.test.gpu_device_name()

import numpy as np
from sklearn.model_selection import KFold
from itertools import product
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Masking, LSTM, GRU
from sklearn.model_selection import KFold
# import early stopping
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import class_weight

from tensorflow.keras.layers import Input, Concatenate, Permute, Reshape, Multiply, Lambda, Add
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def specificity_score(y_true, y_pred):

    # Convert probabilities to binary predictions
    y_pred_bin = np.argmax(y_pred, axis=1)
    y_true_bin = np.argmax(y_true, axis=1)

    tn = np.sum((y_true_bin == 0) & (y_pred_bin == 0))
    fp = np.sum((y_true_bin == 0) & (y_pred_bin != 0))
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    return specificity

def custom_performance_metric(accuracy, specificity, recall, precision, false_positive_rate, accuracy_weight=2):
    """
    Calculate the custom performance metric including precision and with higher weight for accuracy.

    Parameters:
    - accuracy: The accuracy of the model.
    - specificity: The specificity of the model.
    - recall: The recall (sensitivity) of the model.
    - precision: The precision of the model.
    - false_positive_rate: The false positive rate of the model.
    - accuracy_weight: The weight to give to accuracy in the metric calculation.

    Returns:
    - A float representing the custom performance metric.
    """
    # Adjusted calculation to weight accuracy higher
    weighted_accuracy = accuracy * accuracy_weight
    total_weight = accuracy_weight + 1 + 1 + 1  # Adding the implicit weight of 1 for the other metrics
    average_metric = (weighted_accuracy + specificity + recall + precision) / total_weight

    # Subtract the false positive rate
    custom_metric = average_metric - false_positive_rate

    return custom_metric



In [26]:
def train_and_evaluate(dataset_names, create_model_fn, class_weights):

    dataset_results = {}

    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Find best dataset to train and test model on
    #dataset_names = ['X_smoothed_mean_norm']

    for dataset_name in dataset_names:

        # Initialize variables to track the best model
        best_model = None
        best_performance = 0
        best_model_details = ''


         # Load in dataset from pickle
        with open(f'drive/MyDrive/Pickles/{dataset_name}.pickle', 'rb') as handle:
            dataset = pickle.load(handle)

        dataset = np.array(dataset)

        dataset_results[dataset_name] = {}

        print(f'Working on dataset: {dataset_name}')

        accuracies = []
        sensitivities = []
        false_positive_rates = []
        specificities = []
        precisions = []
        performance = []

        fold = 1

        for train_index, test_index in outer_cv.split(dataset):

            # Print current progress
            print(f'Working on fold: {fold}')
            fold += 1

            X_train, X_test = dataset[train_index], dataset[test_index]
            Y_train, Y_test = abnormal_encoded[train_index], abnormal_encoded[test_index]

            X_train = X_train.astype('float32')
            Y_train = Y_train.astype('float32')
            X_test = X_test.astype('float32')
            Y_test = Y_test.astype('float32')

            model = create_model_fn(X_train.shape[1:])
            early_stopping = EarlyStopping(monitor='val_loss', patience=3)

            if class_weights:

                Y_train_classes = np.argmax(Y_train, axis=1)

                # Compute class weights
                cw = class_weight.compute_class_weight('balanced',
                                                    classes=np.unique(Y_train_classes),
                                                    y=Y_train_classes)

                class_weights_dict = dict(enumerate(cw))

                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1, class_weight=class_weights_dict)

            else:
                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

            # Predict the test set
            print('Predicting test set')
            Y_pred = model.predict(X_test)

            Y_pred_classes = np.argmax(Y_pred, axis=1)
            Y_test_classes = np.argmax(Y_test, axis=1)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracy = (accuracy_score(Y_test_classes, Y_pred_classes))
            sensitivity = (recall_score(Y_test_classes, Y_pred_classes, average='macro'))
            false_positive_rate = (1 - specificity_score(Y_test, Y_pred))
            specificity = (specificity_score(Y_test, Y_pred))
            precision = (precision_score(Y_test_classes, Y_pred_classes, average='macro'))

            # Calculate the custom performance metric
            model_performance = custom_performance_metric(accuracy, specificity, sensitivity, precision, false_positive_rate)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            false_positive_rates.append(false_positive_rate)
            specificities.append(specificity)
            precisions.append(precision)
            performance.append(model_performance)

            print(f"Model's custom performance metric: {model_performance}")
            '''
            # Update the best model if current model is better
            if model_performance > best_performance:
                best_performance = model_performance
                best_model = model
                best_model_details = f'{dataset_name}_fold_{fold}'''

        avg_accuracy = np.mean(accuracies)
        avg_sensitivity = np.mean(sensitivities)
        avg_false_positive_rate = np.mean(false_positive_rates)
        avg_specificity = np.mean(specificities)
        avg_precision = np.mean(precisions)
        avg_performance = np.mean(performance)

        std_accuracy = np.std(accuracies)
        std_sensitivity = np.std(sensitivities)
        std_false_positive_rate = np.std(false_positive_rates)
        std_specificity = np.std(specificities)
        std_precision = np.std(precisions)
        std_performance = np.std(performance)

        dataset_results[dataset_name]['Accuracy'] = (avg_accuracy, std_accuracy)
        dataset_results[dataset_name]['Sensitivity'] = (avg_sensitivity, std_sensitivity)
        dataset_results[dataset_name]['False Positive Rate'] = (avg_false_positive_rate, std_false_positive_rate)
        dataset_results[dataset_name]['Specificity'] = (avg_specificity, std_specificity)
        dataset_results[dataset_name]['Precision'] = (avg_precision, std_precision)
        dataset_results[dataset_name]['Performance'] = (avg_performance, std_performance)

        '''
        if best_model:
            model_save_path = f'models/best_model_{best_model_details}.keras'
            print(f'Saving best model to {model_save_path}')
            best_model.save(model_save_path)
            print(f'Best Model Details: {best_model_details}, Performance: {best_performance}')'''

        # Delete dataset to free up memory
        del dataset
        del Y_pred

    for dataset_name, results in dataset_results.items():
        print(f'Dataset: {dataset_name}')
        for metric, (avg, std) in results.items():
            print(f'{metric}: {avg} +/- {std}')
        print('\n')

    return dataset_results

In [20]:
# Model functions

def create_cnn_model(shape, filters=32, kernel_size=3, dropout_rate=0.8):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))  # Adjust the input_shape to match your dataset
    model.add(Conv1D(filters, kernel_size, activation='relu', input_shape=(19301, 16)))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)) )  # Reduced the number of neurons in the dense layer
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(shape, lstm_units=32, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(LSTM(lstm_units, return_sequences=False))  # 'return_sequences=False' because we only need the last output
    model.add(Dropout(dropout_rate))
    model.add(Dense(lstm_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_gru_model(shape, gru_units=32, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(GRU(gru_units, return_sequences=False))  # return_sequences=False because we only need the last output
    model.add(Dropout(dropout_rate))
    model.add(Dense(gru_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_lstm_model(shape, filters=32, kernel_size=3, lstm_units=64, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(lstm_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_gru_model(shape, filters=32, kernel_size=3, gru_units=64, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(GRU(gru_units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(gru_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_attention_model(shape, lstm_units=64, dropout_rate=0.5, output_classes=3):
    inputs = Input(shape=shape)
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention = Dense(1, activation='tanh')(lstm_out)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([lstm_out, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(lstm_units,))(sent_representation)
    dropout = Dropout(dropout_rate)(sent_representation)
    dense = Dense(lstm_units, activation='relu')(dropout)
    outputs = Dense(output_classes, activation='softmax')(dense)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [28]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Multiply, Permute, RepeatVector, Lambda, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def create_cnn_attention_model(shape, filters=32, kernel_size=3, dropout_rate=0.5, output_classes=3):
    inputs = Input(shape=shape)
    conv_out = Conv1D(filters, kernel_size, activation='relu')(inputs)
    conv_out = MaxPooling1D(2)(conv_out)

    # Attention mechanism
    attention_probs = Dense(filters, activation='softmax', name='attention_vec')(conv_out)
    attention_mul = Multiply()([conv_out, attention_probs])

    # Following the attention, flatten to feed into a dense layer
    representation = Flatten()(attention_mul)
    representation = Dropout(dropout_rate)(representation)
    representation = Dense(64, activation='relu')(representation)

    outputs = Dense(output_classes, activation='softmax')(representation)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [5]:
 # Load in dataset from pickle
with open('drive/MyDrive/Pickles/abnormal_encoded.pickle', 'rb') as handle:
    abnormal_encoded = pickle.load(handle)
dataset_results = {'X_smoothed_mean_norm_month': {}, 'X_smoothed_median_norm_month': {}, 'X_smoothed_mean_norm': {}, 'X_smoothed_median_norm': {}}

In [16]:
cnn_dataset_result_cw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_cnn_model, True)


Working on dataset: X_smoothed_mean_norm
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 73ms/step - loss: 64.3880 - accuracy: 0.5686 - val_loss: 51.4076 - val_accuracy: 0.2692
Epoch 2/15
7/7 [==============================] - 0s 31ms/step - loss: 17.2277 - accuracy: 0.6471 - val_loss: 35.9825 - val_accuracy: 0.7308
Epoch 3/15
7/7 [==============================] - 0s 31ms/step - loss: 11.8004 - accuracy: 0.6961 - val_loss: 19.0327 - val_accuracy: 0.6154
Epoch 4/15
7/7 [==============================] - 0s 33ms/step - loss: 6.8147 - accuracy: 0.8529 - val_loss: 29.8845 - val_accuracy: 0.7308
Epoch 5/15
7/7 [==============================] - 0s 30ms/step - loss: 2.4500 - accuracy: 0.9804 - val_loss: 19.6272 - val_accuracy: 0.7308
Epoch 6/15
7/7 [==============================] - 0s 35ms/step - loss: 2.1828 - accuracy: 0.9608 - val_loss: 23.0009 - val_accuracy: 0.6923
Predicting test set
2/2 [==============================] - 0s 8ms/step
Model's cust

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 185ms/step - loss: 47.2806 - accuracy: 0.4854 - val_loss: 9.2705 - val_accuracy: 0.6538
Epoch 2/15
7/7 [==============================] - 0s 32ms/step - loss: 10.0177 - accuracy: 0.6602 - val_loss: 10.9711 - val_accuracy: 0.4615
Epoch 3/15
7/7 [==============================] - 0s 32ms/step - loss: 5.4920 - accuracy: 0.7670 - val_loss: 12.7735 - val_accuracy: 0.7692
Epoch 4/15
7/7 [==============================] - 0s 32ms/step - loss: 5.4368 - accuracy: 0.8738 - val_loss: 17.4143 - val_accuracy: 0.5385
Predicting test set
1/1 [==============================] - 0s 109ms/step
Model's custom performance metric: -0.06999999999999995
Working on fold: 4
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 94ms/step - loss: 56.4266 - accuracy: 0.5146 - val_loss: 117.9981 - val_accuracy: 0.3077
Epoch 2/15
7/7 [==============================] - 0s 40ms/step - loss: 51.5782 - accuracy: 0.5728 - val_loss: 86.2441 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 67ms/step - loss: 44.7753 - accuracy: 0.4272 - val_loss: 2.3558 - val_accuracy: 0.3077
Epoch 2/15
7/7 [==============================] - 0s 34ms/step - loss: 2.6432 - accuracy: 0.4175 - val_loss: 3.0114 - val_accuracy: 0.3077
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 3.1443 - accuracy: 0.4175 - val_loss: 3.3281 - val_accuracy: 0.3077
Epoch 4/15
7/7 [==============================] - 0s 37ms/step - loss: 3.3545 - accuracy: 0.4175 - val_loss: 3.4045 - val_accuracy: 0.3077
Predicting test set
1/1 [==============================] - 0s 89ms/step
Model's custom performance metric: -0.759375
Working on fold: 5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 1s 67ms/step - loss: 54.6418 - accuracy: 0.5146 - val_loss: 37.5831 - val_accuracy: 0.6923
Epoch 2/15
7/7 [==============================] - 0s 38ms/step - loss: 16.0118 - accuracy: 0.6796 - val_loss: 46.2857 - val_accuracy: 0.7692
Epoch 3/15
7/7 [==============================] - 0s 37ms/step - loss: 14.2701 - accuracy: 0.7573 - val_loss: 19.3205 - val_accuracy: 0.6154
Epoch 4/15
7/7 [==============================] - 0s 37ms/step - loss: 4.1637 - accuracy: 0.9126 - val_loss: 33.5229 - val_accuracy: 0.7692
Epoch 5/15
7/7 [==============================] - 0s 32ms/step - loss: 2.0522 - accuracy: 0.9903 - val_loss: 23.9545 - val_accuracy: 0.7308
Epoch 6/15
7/7 [==============================] - 0s 31ms/step - loss: 2.2402 - accuracy: 0.9612 - val_loss: 27.7600 - val_accuracy: 0.7308
Predicting test set
1/1 [==============================] - 0s 90ms/step
Model's custom performance metric: 0.4343807167491378
Saving best model

In [ ]:
cnn_dataset_result_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_cnn_model, False)


In [27]:
improved_cnn_dataset_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_improved_cnn_model, False)

Working on dataset: X_smoothed_mean_norm
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 94ms/step - loss: 27.2243 - accuracy: 0.5294 - val_loss: 35.7406 - val_accuracy: 0.2692
Epoch 2/15
7/7 [==============================] - 0s 32ms/step - loss: 5.9086 - accuracy: 0.7157 - val_loss: 31.1154 - val_accuracy: 0.4231
Epoch 3/15
7/7 [==============================] - 0s 32ms/step - loss: 5.7823 - accuracy: 0.7941 - val_loss: 28.1336 - val_accuracy: 0.4231
Epoch 4/15
7/7 [==============================] - 0s 32ms/step - loss: 5.7945 - accuracy: 0.6765 - val_loss: 21.7747 - val_accuracy: 0.3846
Epoch 5/15
7/7 [==============================] - 0s 32ms/step - loss: 4.9729 - accuracy: 0.6765 - val_loss: 14.3153 - val_accuracy: 0.3846
Epoch 6/15
7/7 [==============================] - 0s 33ms/step - loss: 6.8106 - accuracy: 0.6569 - val_loss: 12.6949 - val_accuracy: 0.3846
Epoch 7/15
7/7 [==============================] - 0s 32ms/step - loss: 5.0421 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 3s 74ms/step - loss: 13.7742 - accuracy: 0.5340 - val_loss: 175.3926 - val_accuracy: 0.0000e+00
Epoch 2/15
7/7 [==============================] - 0s 35ms/step - loss: 19.0590 - accuracy: 0.6990 - val_loss: 192.5296 - val_accuracy: 0.0000e+00
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 16.6453 - accuracy: 0.7087 - val_loss: 174.1127 - val_accuracy: 0.0000e+00
Epoch 4/15
7/7 [==============================] - 0s 32ms/step - loss: 9.8452 - accuracy: 0.7573 - val_loss: 171.4345 - val_accuracy: 0.0000e+00
Epoch 5/15
7/7 [==============================] - 0s 32ms/step - loss: 5.4821 - accuracy: 0.8252 - val_loss: 143.9539 - val_accuracy: 0.0000e+00
Epoch 6/15
7/7 [==============================] - 0s 32ms/step - loss: 4.5153 - accuracy: 0.8641 - val_loss: 111.8414 - val_accuracy: 0.0385
Epoch 7/15
7/7 [==============================] - 0s 32ms/step - loss: 5.3222 - accuracy: 0.8155 - val_loss: 68.1974 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 74ms/step - loss: 19.9224 - accuracy: 0.4660 - val_loss: 27.9593 - val_accuracy: 0.6538
Epoch 2/15
7/7 [==============================] - 0s 37ms/step - loss: 17.8172 - accuracy: 0.6311 - val_loss: 19.0627 - val_accuracy: 0.6538
Epoch 3/15
7/7 [==============================] - 0s 34ms/step - loss: 19.6719 - accuracy: 0.6214 - val_loss: 22.9089 - val_accuracy: 0.7692
Epoch 4/15
7/7 [==============================] - 0s 33ms/step - loss: 7.3610 - accuracy: 0.7282 - val_loss: 29.6084 - val_accuracy: 0.7692
Epoch 5/15
7/7 [==============================] - 0s 33ms/step - loss: 7.9557 - accuracy: 0.6990 - val_loss: 35.2468 - val_accuracy: 0.7308
Predicting test set
1/1 [==============================] - 0s 112ms/step
Model's custom performance metric: 0.4825396825396827
Working on fold: 4


...

Working on dataset: X_smoothed_median_norm_month
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 74ms/step - loss: 27.3661 - accuracy: 0.5000 - val_loss: 76.7458 - val_accuracy: 0.6154
Epoch 2/15
7/7 [==============================] - 0s 33ms/step - loss: 16.2264 - accuracy: 0.5686 - val_loss: 57.0708 - val_accuracy: 0.6538
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 7.8916 - accuracy: 0.7941 - val_loss: 50.5863 - val_accuracy: 0.6538
Epoch 4/15
7/7 [==============================] - 0s 33ms/step - loss: 6.7087 - accuracy: 0.7647 - val_loss: 50.5702 - val_accuracy: 0.6538
Epoch 5/15
7/7 [==============================] - 0s 32ms/step - loss: 7.7800 - accuracy: 0.7353 - val_loss: 50.2155 - val_accuracy: 0.6538
Epoch 6/15
7/7 [==============================] - 0s 33ms/step - loss: 6.9368 - accuracy: 0.7451 - val_loss: 43.1398 - val_accuracy: 0.6538
Epoch 7/15
7/7 [==============================] - 0s 33ms/step - loss: 5.799

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 74ms/step - loss: 20.8033 - accuracy: 0.5437 - val_loss: 31.7035 - val_accuracy: 0.5769
Epoch 2/15
7/7 [==============================] - 0s 37ms/step - loss: 14.3362 - accuracy: 0.5922 - val_loss: 28.0241 - val_accuracy: 0.5769
Epoch 3/15
7/7 [==============================] - 0s 34ms/step - loss: 6.2835 - accuracy: 0.8058 - val_loss: 41.2117 - val_accuracy: 0.5000
Epoch 4/15
7/7 [==============================] - 0s 34ms/step - loss: 6.3975 - accuracy: 0.7184 - val_loss: 44.9140 - val_accuracy: 0.5000
Epoch 5/15
7/7 [==============================] - 0s 35ms/step - loss: 9.6585 - accuracy: 0.7184 - val_loss: 34.3076 - val_accuracy: 0.5000
Predicting test set
1/1 [==============================] - 0s 110ms/step
Model's custom performance metric: 0.2334742351046699
Working on fold: 5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 75ms/step - loss: 31.3694 - accuracy: 0.5825 - val_loss: 17.5378 - val_accuracy: 0.6154
Epoch 2/15
7/7 [==============================] - 0s 35ms/step - loss: 21.9642 - accuracy: 0.6699 - val_loss: 59.4887 - val_accuracy: 0.3462
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 20.3301 - accuracy: 0.6796 - val_loss: 20.3978 - val_accuracy: 0.6538
Epoch 4/15
7/7 [==============================] - 0s 35ms/step - loss: 6.9052 - accuracy: 0.7670 - val_loss: 23.9619 - val_accuracy: 0.7308
Predicting test set
1/1 [==============================] - 0s 107ms/step
Model's custom performance metric: 0.5867712550607287
Dataset: X_smoothed_mean_norm
Accuracy: 0.5032196969696969 +/- 0.07013101834210482
Sensitivity: 0.45610536488682946 +/- 0.14382044690527263
False Positive Rate: 0.28437397842432166 +/- 0.19023883006187628
Specificity: 0.7156260215756783 +/- 0.19023883006187628
Precision: 0.513921500463865 +/- 0.

In [29]:
attention_cnn_dataset_results =  train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_cnn_attention_model, False)

Working on dataset: X_smoothed_mean_norm
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 70ms/step - loss: 1.3095 - accuracy: 0.4902 - val_loss: 0.6852 - val_accuracy: 0.6538
Epoch 2/15
7/7 [==============================] - 0s 33ms/step - loss: 0.3866 - accuracy: 0.7843 - val_loss: 1.5877 - val_accuracy: 0.7308
Epoch 3/15
7/7 [==============================] - 0s 32ms/step - loss: 0.1330 - accuracy: 0.9510 - val_loss: 1.0152 - val_accuracy: 0.6923
Epoch 4/15
7/7 [==============================] - 0s 32ms/step - loss: 0.0459 - accuracy: 1.0000 - val_loss: 1.4661 - val_accuracy: 0.7308
Predicting test set
2/2 [==============================] - 0s 5ms/step
Model's custom performance metric: 0.6496270000514481
Working on fold: 2
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 71ms/step - loss: 1.1824 - accuracy: 0.5049 - val_loss: 0.8554 - val_accuracy: 0.4231
Epoch 2/15
7/7 [==============================] - 0s 35ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Working on dataset: X_smoothed_mean_norm_month
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 73ms/step - loss: 2.1181 - accuracy: 0.4706 - val_loss: 1.1965 - val_accuracy: 0.7308
Epoch 2/15
7/7 [==============================] - 0s 36ms/step - loss: 1.0705 - accuracy: 0.6961 - val_loss: 1.1050 - val_accuracy: 0.6154
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 0.5142 - accuracy: 0.7843 - val_loss: 1.0710 - val_accuracy: 0.6538
Epoch 4/15
7/7 [==============================] - 0s 33ms/step - loss: 0.2693 - accuracy: 0.9020 - val_loss: 0.9226 - val_accuracy: 0.6538
Epoch 5/15
7/7 [==============================] - 0s 33ms/step - loss: 0.1393 - accuracy: 0.9804 - val_loss: 0.7842 - val_accuracy: 0.6538
Epoch 6/15
7/7 [==============================] - 0s 35ms/step - loss: 0.0747 - accuracy: 1.0000 - val_loss: 1.0616 - val_accuracy: 0.6923
Epoch 7/15
7/7 [==============================] - 0s 33ms/step - loss: 0.0466 - accura

In [ ]:
lstm_dataset_results_cw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_lstm_model, True)


In [ ]:
lstm_dataset_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_lstm_model, False)


In [ ]:
gru_dataset_results_cw = c(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_gru_model, True)


In [ ]:
gru_dataset_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_gru_model, False)


In [ ]:
cnn_lstm_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_cnn_lstm_model,False)

In [ ]:
cnn_gru_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_cnn_gru_model,False)

In [ ]:
lstm_attention_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_lstm_attention_model,False)